In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Rondônia - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_RO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego,Rondônia - Consumo de Cimento (t)
0,2003,0.688843,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,59.844000,2.068350e+07,1.587672e+06,11.468162,1.828350e+07,8.226062,185.386000
1,2004,0.691090,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,59.844000,2.100894e+07,1.609824e+06,11.519650,1.840022e+07,8.152626,209.004000
2,2005,0.693216,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,59.844000,2.130613e+07,1.615999e+06,11.569863,1.848933e+07,8.130621,233.057000
3,2006,0.695228,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,59.844000,2.157640e+07,1.633992e+06,11.602119,1.855816e+07,8.038397,272.001000
4,2007,0.697131,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,64.041487,2.184313e+07,1.630117e+06,11.647841,1.864093e+07,8.013839,351.377000
5,2008,0.698932,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,150.448962,2.208447e+07,1.637885e+06,11.689351,1.870941e+07,7.979888,390.788000
6,2009,0.700638,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,336.177608,2.226473e+07,1.643078e+06,11.684640,1.873529e+07,8.008176,452.325000
7,2010,0.702254,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,502.201514,2.285838e+07,1.640521e+06,11.724602,1.891099e+07,7.971248,921.425000
8,2011,0.703786,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,583.009861,2.338128e+07,1.618454e+06,11.764179,1.903017e+07,7.956614,1021.570000
9,2012,0.705401,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,560.443805,2.383098e+07,1.560372e+06,11.757682,1.910530e+07,6.546489,933.501000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,0.688843,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,59.844000,2.068350e+07,1.587672e+06,11.468162,1.828350e+07,8.226062
1,0.691090,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,59.844000,2.100894e+07,1.609824e+06,11.519650,1.840022e+07,8.152626
2,0.693216,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,59.844000,2.130613e+07,1.615999e+06,11.569863,1.848933e+07,8.130621
3,0.695228,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,59.844000,2.157640e+07,1.633992e+06,11.602119,1.855816e+07,8.038397
4,0.697131,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,64.041487,2.184313e+07,1.630117e+06,11.647841,1.864093e+07,8.013839
5,0.698932,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,150.448962,2.208447e+07,1.637885e+06,11.689351,1.870941e+07,7.979888
6,0.700638,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,336.177608,2.226473e+07,1.643078e+06,11.684640,1.873529e+07,8.008176
7,0.702254,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,502.201514,2.285838e+07,1.640521e+06,11.724602,1.891099e+07,7.971248
8,0.703786,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,583.009861,2.338128e+07,1.618454e+06,11.764179,1.903017e+07,7.956614
9,0.705401,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,560.443805,2.383098e+07,1.560372e+06,11.757682,1.910530e+07,6.546489


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      209.004000
1      233.057000
2      272.001000
3      351.377000
4      390.788000
5      452.325000
6      921.425000
7     1021.570000
8      933.501000
9      826.670000
10     614.394714
11     510.273771
12     410.580514
13     406.814000
14     376.138000
15     399.441000
16     488.194000
17     502.824000
18     501.142000
19            NaN
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,-1.768808,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.149207,-1.622684,0.384181,-2.216191,-1.841163,1.166353
1,-1.495156,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.149207,-1.414824,0.683585,-1.737423,-1.507080,1.018944
2,-1.236203,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.149207,-1.225013,0.767051,-1.270509,-1.252031,0.974772
3,-0.991191,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-1.149207,-1.052390,1.010242,-0.970566,-1.055042,0.789652
4,-0.759364,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-1.125291,-0.882035,0.957865,-0.545420,-0.818130,0.740357
5,-0.539966,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.632962,-0.727890,1.062859,-0.159427,-0.622119,0.672206
6,-0.332239,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,0.425273,-0.612763,1.133050,-0.203230,-0.548059,0.728989
7,-0.135428,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,1.371236,-0.233603,1.098488,0.168358,-0.045155,0.654863
8,0.051225,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,1.831662,0.100371,0.800233,0.536376,0.295942,0.625489
9,0.247948,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,1.703087,0.387593,0.015197,0.475959,0.510983,-2.205064


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      209.004000
1      233.057000
2      272.001000
3      351.377000
4      390.788000
5      452.325000
6      921.425000
7     1021.570000
8      933.501000
9      826.670000
10     614.394714
11     510.273771
12     410.580514
13     406.814000
14     376.138000
15     399.441000
16     488.194000
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
17,0.923679,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-0.082464,0.874068,-0.785213,0.698552,0.753401,-1.482458


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    502.824
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [13]:
train_input.iloc[-(len(train_input)//5):]

,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
14,1.405167,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,-0.197348,1.243290,-1.413734,1.196729,1.193375,-0.728473
15,1.337642,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,-0.257552,1.198503,-1.288925,1.145168,1.080670,-0.989258
16,1.164322,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,-0.161984,1.099624,-1.061532,0.950531,0.962531,-1.286188


In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2261187251, 2609380172, 2407841163, 2033908010, 603250554, 1704322577, 3792381065, 3747715919, 477230950, 2695766801, 73660621, 2336710799, 3197455411, 713974598, 3035513969, 795081655, 3364328352, 1931253620, 3710201704, 70190091, 2406903439, 2286222627, 3803078978, 3991342659, 1218414694, 985958149, 3559416335, 1084712013, 3344960532, 1637449337, 3577852221, 3727659059, 3063844718, 463018339, 3845019406, 1221283653, 3391503037, 3993456108, 3394061304, 1669506754, 711673521, 4168751081, 634791442, 470528718, 88705992, 3976260813, 3770374730, 2910195734, 1533663867, 546746875, 193337911, 712628696, 2655257931, 1680978104, 3807653469, 1855124224, 22999883, 3029439839, 1707117361, 562209655, 755554631, 173332959, 879709625, 1704387112, 1755708088, 1782310376, 3895812529, 2935819250, 2929114786, 689963769, 2025462623, 2058701082, 1345909015, 1641937654, 3652143621, 657567177, 3874364025, 3481334234, 2011042699, 3856424700, 2382340333, 103089121, 3219133360, 243622934, 2316551445, 2712467

val_loss: 3655.2021484375


Step: 87 ___________________________________________
val_loss: 6078.85595703125


Step: 88 ___________________________________________
val_loss: 7927.1796875


Step: 89 ___________________________________________
val_loss: 3259.234130859375


Step: 90 ___________________________________________
val_loss: 17388.390625


Step: 91 ___________________________________________
val_loss: 43733.83203125


Step: 92 ___________________________________________
val_loss: 6359.41015625


Step: 93 ___________________________________________
val_loss: 2330.757080078125


Step: 94 ___________________________________________
val_loss: 8595.9873046875


Step: 95 ___________________________________________
val_loss: 23640.509765625


Step: 96 ___________________________________________
val_loss: 2356.678955078125


Step: 97 ___________________________________________
val_loss: 2332.550048828125


Step: 98 ___________________________________________
val_loss: 2338.7353515625




In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 438ms/step - loss: 329942.0625 - val_loss: 160114.6562
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 311679.2500 - val_loss: 84156.7734
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 147628.7031 - val_loss: 4758.0488
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 345306.8438 - val_loss: 74603.9375
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 289787.5000 - val_loss: 27097.4434
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 206947.8438 - val_loss: 32319.4746
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 116224.3672 - val_loss: 180247.5781
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 213459.5156 - val_loss: 24171.2949
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 276478.3750 - val_loss: 87975.7891
Epoch 10/10000
1/

Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 41047.5547 - val_loss: 16168.8906
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 45609.3633 - val_loss: 7983.0269
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 51571.9727 - val_loss: 20968.5762
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 36538.7695 - val_loss: 14561.3779
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 35664.3398 - val_loss: 25882.6250
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 42089.9219 - val_loss: 17509.9844
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 31389.1953 - val_loss: 16748.0215
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 30727.4551 - val_loss: 11621.2109
Epoch 84/10000
1/1 [==============================] - 0s 13ms/step - loss: 28189.8438 - val_loss: 11990.1045
Epoch 85/10000
1/1 [

1/1 [==============================] - 0s 13ms/step - loss: 15832.6953 - val_loss: 9615.2314
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 12889.1543 - val_loss: 11089.2461
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 18704.9844 - val_loss: 4079.9182
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 20515.3281 - val_loss: 11094.5039
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 11538.5664 - val_loss: 4624.0991
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 17845.0703 - val_loss: 9973.1992
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 10613.8057 - val_loss: 4361.7588
Epoch 158/10000
1/1 [==============================] - 0s 13ms/step - loss: 13963.8652 - val_loss: 6909.0728
Epoch 159/10000
1/1 [==============================] - 0s 13ms/step - loss: 11804.4795 - val_loss: 4689.7729
Epoch 160/10000
1/1 [============

1/1 [==============================] - 0s 13ms/step - loss: 12514.0049 - val_loss: 6288.9565
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 12566.4980 - val_loss: 2332.9717
Epoch 228/10000
1/1 [==============================] - 0s 14ms/step - loss: 6844.6865 - val_loss: 2336.4656
Epoch 229/10000
1/1 [==============================] - 0s 14ms/step - loss: 4963.4165 - val_loss: 2375.8149
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 7135.3589 - val_loss: 2439.7117
Epoch 231/10000
1/1 [==============================] - 0s 13ms/step - loss: 11245.1523 - val_loss: 2567.9729
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 10880.8232 - val_loss: 2334.0820
Epoch 233/10000
1/1 [==============================] - 0s 13ms/step - loss: 4845.0269 - val_loss: 2535.0872
Epoch 234/10000
1/1 [==============================] - 0s 13ms/step - loss: 6280.1426 - val_loss: 2334.7639
Epoch 235/10000
1/1 [===================

Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 9682.0391 - val_loss: 3914.0449
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 9569.3311 - val_loss: 2754.8132
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 4207.4951 - val_loss: 2408.9756
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 7238.3784 - val_loss: 2523.6777
Epoch 306/10000
1/1 [==============================] - 0s 13ms/step - loss: 7708.7607 - val_loss: 2811.5286
Epoch 307/10000
1/1 [==============================] - 0s 13ms/step - loss: 6803.3174 - val_loss: 2564.0347
Epoch 308/10000
1/1 [==============================] - 0s 13ms/step - loss: 4877.7515 - val_loss: 2382.8018
Epoch 309/10000
1/1 [==============================] - 0s 13ms/step - loss: 7896.8906 - val_loss: 5740.0513
Epoch 310/10000
1/1 [==============================] - 0s 13ms/step - loss: 9262.5547 - val_loss: 16120.1055
Epoch 311/10000
1/1 [======

...

1/1 [==============================] - 0s 13ms/step - loss: 10401.4268 - val_loss: 9828.9473
Epoch 757/10000
1/1 [==============================] - 0s 13ms/step - loss: 10868.2236 - val_loss: 8529.3369
Epoch 758/10000
1/1 [==============================] - 0s 13ms/step - loss: 10686.8662 - val_loss: 8904.0107
Epoch 759/10000
1/1 [==============================] - 0s 13ms/step - loss: 10526.8076 - val_loss: 9696.6982
Epoch 760/10000
1/1 [==============================] - 0s 13ms/step - loss: 10699.6689 - val_loss: 8770.7432
Epoch 761/10000
1/1 [==============================] - 0s 13ms/step - loss: 10808.5195 - val_loss: 8744.0771
Epoch 762/10000
1/1 [==============================] - 0s 13ms/step - loss: 10673.9453 - val_loss: 8933.7500
Epoch 763/10000
1/1 [==============================] - 0s 13ms/step - loss: 10669.8047 - val_loss: 8906.9551
Epoch 764/10000
1/1 [==============================] - 0s 13ms/step - loss: 10687.5098 - val_loss: 8613.7939
Epoch 765/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 11109.9150 - val_loss: 8248.3486
Epoch 832/10000
1/1 [==============================] - 0s 14ms/step - loss: 10445.5312 - val_loss: 9464.5752
Epoch 833/10000
1/1 [==============================] - 0s 13ms/step - loss: 10596.8281 - val_loss: 8995.7861
Epoch 834/10000
1/1 [==============================] - 0s 13ms/step - loss: 10141.3613 - val_loss: 10558.0732
Epoch 835/10000
1/1 [==============================] - 0s 13ms/step - loss: 10848.3789 - val_loss: 8389.9043
Epoch 836/10000
1/1 [==============================] - 0s 14ms/step - loss: 10434.4346 - val_loss: 9493.1943
Epoch 837/10000
1/1 [==============================] - 0s 13ms/step - loss: 10668.4258 - val_loss: 9611.9775
Epoch 838/10000
1/1 [==============================] - 0s 13ms/step - loss: 11129.2881 - val_loss: 8148.3672
Epoch 839/10000
1/1 [==============================] - 0s 13ms/step - loss: 11255.8164 - val_loss: 8090.4922
Epoch 840/10000
1/1 [=============

1/1 [==============================] - 0s 13ms/step - loss: 11762.5225 - val_loss: 7052.9604
Epoch 907/10000
1/1 [==============================] - 0s 13ms/step - loss: 12027.6523 - val_loss: 6867.3882
Epoch 908/10000
1/1 [==============================] - 0s 13ms/step - loss: 8354.1172 - val_loss: 6204.3320
Epoch 909/10000
1/1 [==============================] - 0s 13ms/step - loss: 11979.5977 - val_loss: 7143.6816
Epoch 910/10000
1/1 [==============================] - 0s 13ms/step - loss: 8269.6797 - val_loss: 6496.8691
Epoch 911/10000
1/1 [==============================] - 0s 13ms/step - loss: 12434.4863 - val_loss: 8246.1230
Epoch 912/10000
1/1 [==============================] - 0s 13ms/step - loss: 11700.0791 - val_loss: 7186.8110
Epoch 913/10000
1/1 [==============================] - 0s 13ms/step - loss: 11651.5371 - val_loss: 7388.2642
Epoch 914/10000
1/1 [==============================] - 0s 13ms/step - loss: 11869.7471 - val_loss: 7159.0513
Epoch 915/10000
1/1 [================

1/1 [==============================] - 0s 13ms/step - loss: 8064.6489 - val_loss: 6974.4004
Epoch 982/10000
1/1 [==============================] - 0s 13ms/step - loss: 11626.3047 - val_loss: 8439.7158
Epoch 983/10000
1/1 [==============================] - 0s 13ms/step - loss: 11558.2012 - val_loss: 8220.7822
Epoch 984/10000
1/1 [==============================] - 0s 14ms/step - loss: 11554.1699 - val_loss: 8271.4805
Epoch 985/10000
1/1 [==============================] - 0s 13ms/step - loss: 11551.7129 - val_loss: 8280.3564
Epoch 986/10000
1/1 [==============================] - 0s 13ms/step - loss: 8195.1709 - val_loss: 8877.1377
Epoch 987/10000
1/1 [==============================] - 0s 14ms/step - loss: 11657.4707 - val_loss: 7763.0386
Epoch 988/10000
1/1 [==============================] - 0s 13ms/step - loss: 11601.6631 - val_loss: 8017.0767
Epoch 989/10000
1/1 [==============================] - 0s 13ms/step - loss: 11588.3262 - val_loss: 8007.9863
Epoch 990/10000
1/1 [================

Epoch 1056/10000
1/1 [==============================] - 0s 15ms/step - loss: 11570.6074 - val_loss: 8711.0088
Epoch 1057/10000
1/1 [==============================] - 0s 13ms/step - loss: 7709.3291 - val_loss: 7067.6895
Epoch 1058/10000
1/1 [==============================] - 0s 13ms/step - loss: 11564.9521 - val_loss: 8755.3926
Epoch 1059/10000
1/1 [==============================] - 0s 13ms/step - loss: 11476.2607 - val_loss: 8487.3613
Epoch 1060/10000
1/1 [==============================] - 0s 13ms/step - loss: 7659.2695 - val_loss: 7154.7988
Epoch 1061/10000
1/1 [==============================] - 0s 13ms/step - loss: 7590.8389 - val_loss: 7359.9253
Epoch 1062/10000
1/1 [==============================] - 0s 14ms/step - loss: 7574.3643 - val_loss: 7322.9160
Epoch 1063/10000
1/1 [==============================] - 0s 14ms/step - loss: 11521.5186 - val_loss: 8775.7451
Epoch 1064/10000
1/1 [==============================] - 0s 13ms/step - loss: 11457.8369 - val_loss: 8547.2539
Epoch 1065/100

In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[502.824](test_target) - [[466.35632]](prediction) = 36.46767675781251


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,-1.611543,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-0.647106,-1.539414,-2.086289,-1.729822,-1.637428,1.850658
1,-1.099651,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-0.647106,-1.048104,-0.844413,-1.116076,-1.015841,1.006286
2,-0.615254,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.647106,-0.599454,-0.498210,-0.517525,-0.541303,0.753271
3,-0.156936,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.647106,-0.191432,0.510500,-0.133020,-0.174792,-0.307113
4,0.276717,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.620394,0.211228,0.293250,0.411988,0.266001,-0.589476
5,0.687123,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.070521,0.575576,0.728747,0.906803,0.630695,-0.979847
6,1.075695,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.111404,0.847697,1.019887,0.850652,0.768489,-0.654591
7,1.443849,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,2.167934,1.743903,0.876528,1.327000,1.704180,-1.079188


train_target:


,Rondônia - Consumo de Cimento (t)
0,209.004
1,233.057
2,272.001
3,351.377
4,390.788
5,452.325
6,921.425
7,1021.570


test_input:


...

,Rondônia - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
17,0.923679,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-0.082464,0.874068,-0.785213,0.698552,0.753401,-1.482458


test_target:


,Rondônia - Consumo de Cimento (t)
17,502.824


1/1 [==============================] - 0s 34ms/step
Error: 36.46767675781251




[753.80419921875,
 749.77099609375,
 900.8115234375,
 708.5224609375,
 704.2713012695312,
 698.4212646484375,
 457.85235595703125,
 557.3424072265625,
 452.43994140625,
 466.3563232421875]

In [22]:
display(targets)
display(predictions)

[933.501,
 826.67,
 614.3947142857144,
 510.2737714285714,
 410.5805142857144,
 406.814,
 376.138,
 399.441,
 488.194,
 502.824]

[753.80419921875,
 749.77099609375,
 900.8115234375,
 708.5224609375,
 704.2713012695312,
 698.4212646484375,
 457.85235595703125,
 557.3424072265625,
 452.43994140625,
 466.3563232421875]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

163.83968535156248

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.2995881301718091